In [9]:
import psycopg2
import random
import string
from faker import Faker
import numpy as np
from tqdm import tqdm

conn = psycopg2.connect(database="studs", user="s311745",
                        password="IpcU3kSHTeN94A3b", host="localhost", port="5432")
cur = conn.cursor()
list_table = ["Human", "Employee", "Supplier", "Research_center",
              "Map", "Shots_of_the_terrain", "Shots_to_Employee", "Analysis_of_deposits", "Map_to_Employee", "Area",
              "Drilling_wells"]
list_enum = ['app_enum', 'post_level_enum', 'level_enum',
             'prof_enum', 'type_enum', 'category_enum']
list_proc = ['input_wells','input_analysis','input_area', 'input_map']
cur.execute("DROP FUNCTION certain_area_info(category_enum);")
cur.execute("DROP TABLE IF EXISTS {table}".format(table=', '.join(list_table)))
cur.execute("DROP PROCEDURE IF EXISTS {table}".format(table=', '.join(list_proc)))
cur.execute("DROP TYPE IF EXISTS {table}".format(table=', '.join(list_enum)))
conn.commit()
# create tables
cur.execute("CREATE TYPE app_enum AS ENUM('employee', 'supplier')")
cur.execute(
    "CREATE TYPE post_level_enum AS ENUM('intern', 'junior', 'middle','senior')")
cur.execute("CREATE TYPE level_enum AS ENUM('guest', 'low', 'middle','high')")
cur.execute(
    "CREATE TYPE prof_enum AS ENUM('geologist', 'geochemist', 'geoanalyst')")
cur.execute("CREATE TYPE type_enum AS ENUM('geological', 'geochemical')")
cur.execute(
    "CREATE TYPE category_enum AS ENUM('disapproved','P1', 'P2', 'P3','C2','C1','B','A')")

cur.execute("""
CREATE TABLE Human
(
    human_id    INTEGER PRIMARY KEY,
    human_name  varchar(50) NOT NULL,
    appointment app_enum
);
""")
cur.execute("""
CREATE TABLE Employee
(
    employee_id  INTEGER PRIMARY KEY,
    profession   prof_enum       NOT NULL,
    post         post_level_enum NOT NULL,
    access_level level_enum      NOT NULL,
    human_id     INTEGER REFERENCES Human (human_id)
 
);
""")
cur.execute("""
CREATE TABLE Supplier
(
    supplier_id INTEGER PRIMARY KEY,
    company     varchar(50),
    contacts    varchar(200),
    human_id    INTEGER REFERENCES Human (human_id)
);
""")
cur.execute("""
CREATE TABLE Research_center
(
    research_center_id INTEGER PRIMARY KEY,
    center_name        varchar(100),
    address            varchar(200)
);
""")
cur.execute("""
CREATE TABLE Map
(
    map_id             INTEGER PRIMARY KEY,
    TYPE               type_enum,
    accuracy           varchar(50) NOT NULL,
    scale              varchar(50) NOT NULL,
    datetime           DATE        NOT NULL,
    research_center_id INTEGER REFERENCES Research_center (research_center_id)
 
);
""")
cur.execute("""
CREATE TABLE Shots_of_the_terrain
(
    shots_id         INTEGER PRIMARY KEY,
    accuracy         varchar(50) NOT NULL,
    resolution       varchar(20) NOT NULL,
    datetime         DATE        NOT NULL,
    supplier_id      INTEGER REFERENCES Supplier (supplier_id),
    geologist_map_id INTEGER REFERENCES Map (map_id)
);
""")
cur.execute("""
CREATE TABLE Shots_to_Employee
(
    shots_id    INTEGER REFERENCES Shots_of_the_terrain (shots_id),
    employee_id INTEGER REFERENCES Employee (employee_id)
);
 
""")
cur.execute("""
CREATE TABLE Area
(
    area_id            INTEGER PRIMARY KEY,
    map_id             INTEGER REFERENCES Map (map_id),
    coordinates        varchar(100) NOT NULL,
    area_category      category_enum,
    research_center_id INTEGER REFERENCES Research_center (research_center_id)
);
""")
cur.execute("""
CREATE TABLE Drilling_wells
(
    drilling_wells_id    INTEGER PRIMARY KEY,
    things_structure     varchar(200),
    morphology_of_bodies varchar(200),
    area_id              INTEGER REFERENCES Area (area_id),
    employee_id          INTEGER REFERENCES Employee (employee_id)
);
""")
cur.execute("""
CREATE TABLE Analysis_of_deposits
(
    analyse_id  INTEGER PRIMARY KEY,
    report      varchar(1000),
    estimation  varchar(500),
    losses      varchar(500),
    employee_id INTEGER REFERENCES Employee (employee_id)
);
""")
cur.execute("""
CREATE TABLE Map_to_Employee
(
    map_id      INTEGER REFERENCES Map (map_id),
    employee_id INTEGER REFERENCES Employee (employee_id)
);
""")


cur.execute("""
CREATE OR REPLACE FUNCTION area_info(area_in integer) RETURNS text AS
$$
declare
    temp_coord    varchar;
    temp_category varchar;
    temp_center   varchar;
BEGIN
    select center_name
    into temp_center
    from Research_center
             join Area A on Research_center.research_center_id = A.research_center_id
    where area_id = area_in;
    select coordinates into temp_coord from Area where area_id = area_in;
    select area_category into temp_category from Area where area_id = area_in;

    return format(
            'Area id - %s \n Coordinates - %s \n Category - %s \n Center name - %s',
            area_in, temp_coord, temp_category, temp_center
        );
END;
$$ LANGUAGE plpgsql;
""")

cur.execute("""
    CREATE OR REPLACE FUNCTION certain_area_info(category category_enum)
    RETURNS TABLE
            (
                area_id_     integer,
                center_name_ varchar,
                coordinates_ varchar
            )
AS
$$
BEGIN
    RETURN QUERY select area_id, center_name, coordinates
                 from Research_center
                          join Area A on Research_center.research_center_id = A.research_center_id
                 where A.area_category = category;
END;
$$
    LANGUAGE 'plpgsql';
""")

cur.execute('''
CREATE OR REPLACE PROCEDURE input_wells(things_structure_in varchar, morphology_of_bodies_in varchar, area_in integer,
                                        employee integer, category category_enum)
    LANGUAGE plpgsql AS
$$
declare
    temp_wells_id integer;
BEGIN
    SELECT MAX(drilling_wells_id)
    into temp_wells_id
    FROM Drilling_wells;
    temp_wells_id = temp_wells_id + 1;
    if category <> 'P1' and category <> 'P2' and category <> 'disapproved'
    then
        RAISE EXCEPTION 'Wrong category';
    end if;
    insert into Drilling_wells (drilling_wells_id, things_structure, morphology_of_bodies, area_id, employee_id)
    VALUES (temp_wells_id, things_structure_in, morphology_of_bodies_in, area_in, employee);
    update Area set area_category = category where area_id = area_in;
END
$$;
''')
cur.execute('''
CREATE OR REPLACE PROCEDURE input_analysis(report_in varchar, estimation_in varchar, losses_in varchar, area_in integer,
                                           employee integer, category category_enum)
    LANGUAGE plpgsql AS
$$
declare
    temp_analyse_id integer;
BEGIN
    SELECT MAX(analyse_id)
    into temp_analyse_id
    FROM Analysis_of_deposits;
    temp_analyse_id = temp_analyse_id + 1;
    if category <> 'C1' and category <> 'C2' and category <> 'disapproved' and category <> 'A' and category <> 'B'
    then
        RAISE EXCEPTION 'Wrong category';
    end if;
    insert into Analysis_of_deposits (analyse_id, report, estimation, losses, employee_id)
    VALUES (temp_analyse_id, report_in, estimation_in, losses_in, employee);
    update Area set area_category = category where area_id = area_in;
END
$$;
''')

cur.execute('''CREATE OR REPLACE PROCEDURE input_area(map_id_in integer, coord_in varchar, center_id integer, category category_enum)
    LANGUAGE plpgsql AS
$$
declare
    temp_area_id integer;
BEGIN
    SELECT MAX(area_id)
    into temp_area_id
    FROM Area;
    temp_area_id = temp_area_id + 1;
    if category <> 'P3' and category <> 'P2' and category <> 'disapproved'
    then
        RAISE EXCEPTION 'Wrong category';
    end if;
    insert into Area (area_id, map_id, coordinates, area_category, research_center_id)
    VALUES (temp_area_id,map_id_in,coord_in,category,center_id);
END
$$;''')

cur.execute('''
CREATE OR REPLACE PROCEDURE input_map(type_in type_enum, accuracy_in varchar, scale_in varchar, datetime_in date,
                                      center_id integer, employee_id_in integer)
    LANGUAGE plpgsql AS
$$
declare
    temp_map_id integer;
BEGIN
    SELECT MAX(map_id)
    into temp_map_id
    FROM Map;
    temp_map_id = temp_map_id + 1;
    insert into Map (map_id, type, accuracy, scale, datetime, research_center_id)
    VALUES (temp_map_id, type_in, accuracy_in, scale_in, datetime_in, center_id);
    insert into Map_to_Employee (map_id, employee_id) VALUES (temp_map_id, employee_id_in);
END
$$;
''')

cur.execute('''
create or replace function analyst_tr()
    returns trigger
as
$$
BEGIN
    if (select profession from Employee e where e.employee_id = new.employee_id) <> 'geoanalyst'
    then
        RAISE EXCEPTION 'It is not geoanalyst';
    end if;
    return new;
END;
$$ language plpgsql;
''')
cur.execute('''
create or replace function map_tr()
    returns trigger
as
$$
declare
    area_type type_enum;
BEGIN
    select type into area_type from Map where map_id = new.map_id;
    if (area_type = 'geological' and
        (select profession
         from Employee e
         where e.employee_id = new.employee_id) <> 'geologist')
        or (area_type = 'geochemical' and
            (select profession
             from Employee e
             where e.employee_id = new.employee_id) <>
            'geochemist')
    then
        RAISE EXCEPTION 'Mismatch between Map type and Employee profession';
    end if;
    return new;
END;
$$ language plpgsql;
''')
cur.execute('''
create or replace trigger check_analyst
    before insert
    on analysis_of_deposits
    for each row
execute procedure analyst_tr();
''')
cur.execute('''
create or replace trigger check_drilling
    before insert
    on drilling_wells
    for each row
execute procedure analyst_tr();
''')
cur.execute('''
create or replace trigger check_map
    before insert
    on Map_to_Employee
    for each row
execute procedure map_tr();
''')

conn.commit()

# delect data from tables
cur.execute("TRUNCATE TABLE {table}".format(table=', '.join(list_table)))
conn.commit()

num_data = 10001
list_create_data = ["Human", "Employee", "Supplier", "Research_center",
                    "Map", "Shots_of_the_terrain", "Shots_to_Employee", "Analysis_of_deposits", "Map_to_Employee",
                    "Area", "Drilling_wells"]
fake = Faker()

app_enum = ["employee", "supplier"]
post_level_enum = ["intern", "junior", "middle", "senior"]
level_enum = ["guest", "low", "middle", "high"]
prof_enum = ["geologist", "geochemist", "geoanalyst"]
type_enum = ["geological", "geochemical"]
category_enum = ["P1", "P2", "P3", "C2", "C1", "B", "A"]


def random_id():
    return str(random.randint(1, 100))


def random_date():
    return str(random.randint(1900, 2022)) + '-' + str(random.randint(1, 1)) + \
        '-' + str(random.randint(1, 28))
xrange = list(map(int,np.arange(1, num_data)))
human_id = employee_id = employee_id = supplier_id = Research_center_id = Map_id = Shots_of_the_terrain_id = Analysis_of_deposits_id = area_id = Drilling_wells_id = xrange
np.random.shuffle(human_id)
np.random.shuffle(employee_id)
np.random.shuffle(supplier_id)
np.random.shuffle(Research_center_id)
np.random.shuffle(Map_id)
np.random.shuffle(Shots_of_the_terrain_id)
np.random.shuffle(Analysis_of_deposits_id)
np.random.shuffle(area_id)
np.random.shuffle(Drilling_wells_id)

# 5000
human_num = num_data
# 4000
employee_num = round(0.8 * human_num)
# 1000
supplier_num = round(0.2 * human_num)
# 5000
Shots_of_the_terrain_num = num_data
# 1000
Map_num = round(0.2 * Shots_of_the_terrain_num)
# 4000
area_num =round(0.8 * Shots_of_the_terrain_num)
# 1500
Research_center_id_num = round(0.3 * num_data)
# 4000
Drilling_wells_id_num = round(area_num)
# 4000
Analysis_of_deposits_id_num = area_num
# 1000
Map_to_employee_num = Map_num
# 5000
Shots_to_Employee_num = Shots_of_the_terrain_num

human_arr = []
employee_arr = []
supplier_arr = []
Research_center_id_arr = []
Map_arr = []
Shots_of_the_terrain_arr = []
Analysis_of_deposits_id_arr = []
area_arr = []


for i in tqdm(range(human_num-1)):
        human_arr.append(human_id[i])
        cur.execute("INSERT INTO Human (human_id, human_name, appointment ) VALUES (%s,%s,%s)",
                    (human_id[i], fake.name(), random.choice(app_enum)))
for i in tqdm(range(employee_num)):
        employee_arr.append(employee_id[i])
        arr = cur.execute('''select human_id from Human where appointment = \'employee\'''')
        arr = cur.fetchall()
        cur.execute(
            "INSERT INTO Employee (employee_id, profession, post, access_level, human_id) VALUES (%s,%s,%s,%s,%s)",
            (employee_id[i], random.choice(prof_enum), random.choice(post_level_enum), random.choice(level_enum),
             random.choice(arr)))
for i in tqdm(range(supplier_num)):
        supplier_arr.append(supplier_id[i])
        arr = cur.execute('''select human_id from Human where appointment = \'supplier\'''')
        arr = cur.fetchall()
        cur.execute(
            "INSERT INTO Supplier (supplier_id , company, contacts, human_id) VALUES (%s,%s,%s,%s)",
            (supplier_id[i], fake.name() + " Company", fake.phone_number(), random.choice(arr)))
for i in tqdm(range(Research_center_id_num)):
        Research_center_id_arr.append(Research_center_id[i])
        cur.execute(
            "INSERT INTO Research_center (research_center_id, center_name, address) VALUES (%s,%s,%s)",
            (Research_center_id[i], fake.name() + " Center", fake.address()))
for i in tqdm(range(Map_num)):
        Map_arr.append(Map_id[i])
        cur.execute(
            "INSERT INTO Map (map_id, TYPE, accuracy,scale, datetime,research_center_id ) VALUES (%s,%s,%s,%s,%s,"
            "%s)",
            (Map_id[i], random.choice(type_enum), '1:1', '1980X1024', random_date(), random.choice(Research_center_id_arr)))
for i in tqdm(range(Shots_of_the_terrain_num-1)):
        Shots_of_the_terrain_arr.append(Shots_of_the_terrain_id[i])
        cur.execute(
            "INSERT INTO Shots_of_the_terrain (shots_id, accuracy, resolution,datetime, "
            "supplier_id,geologist_map_id) VALUES (%s,%s,%s,%s,%s,%s)",
            (Shots_of_the_terrain_id[i], '1:1', '1980X1024', random_date(), random.choice(supplier_arr), random.choice(Map_arr)))
for i in tqdm(range(Shots_to_Employee_num)):
        cur.execute(
            "INSERT INTO Shots_to_Employee (shots_id, employee_id) VALUES (%s,%s)",
            (random.choice(Shots_of_the_terrain_arr), random.choice(employee_arr)))
for i in tqdm(range(Analysis_of_deposits_id_num)):
        Analysis_of_deposits_id_arr.append(Analysis_of_deposits_id[i])
        arr = cur.execute('''select employee_id from Employee where profession = \'geoanalyst\'''')
        arr = cur.fetchall()
        cur.execute(
            "INSERT INTO Analysis_of_deposits (analyse_id  , report, estimation  , losses      , employee_id ) "
            "VALUES (%s,%s,%s,%s,%s)",
            (Analysis_of_deposits_id[i], fake.text(), fake.text(), i, random.choice(arr)))
for i in tqdm(range(Map_to_employee_num)):
        arr_geolog = cur.execute('''select employee_id from Employee where profession = \'geologist\'''')
        arr_geolog = cur.fetchall()
        arr_geochem = cur.execute('''select employee_id from Employee where profession = \'geochemist\'''')
        arr_geochem = cur.fetchall()
        map_geochem = cur.execute('''select map_id from Map where type = \'geochemical\'''')
        map_geochem = cur.fetchall()
        map_geolog = cur.execute('''select map_id from Map where type = \'geological\'''')
        map_geolog = cur.fetchall()
        if i % 2 == 0:
            cur.execute(
                "INSERT INTO Map_to_Employee (map_id      , employee_id ) VALUES (%s,%s)",
                (random.choice(map_geolog), random.choice(arr_geolog)))
        else:
            cur.execute(
                "INSERT INTO Map_to_Employee (map_id      , employee_id ) VALUES (%s,%s)",
                (random.choice(map_geochem), random.choice(arr_geochem)))
for i in tqdm(range(area_num)):
        area_arr.append(area_id[i])
        cur.execute(
            "INSERT INTO Area (area_id              , map_id             , coordinates          , area_category   "
            "         , research_center_id  ) VALUES (%s,%s,%s,%s,%s)",
            (area_id[i], random.choice(Map_arr), fake.coordinate(), random.choice(category_enum), random.choice(Research_center_id_arr)))
for i in tqdm(range(Drilling_wells_id_num)):
        arr = cur.execute('''select employee_id from Employee where profession = \'geoanalyst\'''')
        arr = cur.fetchall()
        cur.execute(
            "INSERT INTO Drilling_wells (drilling_wells_id                  , things_structure             , "
            "morphology_of_bodies           , area_id                          , employee_id            ) VALUES "
            "(%s,%s,%s,%s,%s)",
            (Drilling_wells_id[i], fake.text(), fake.text(), random.choice(area_arr), random.choice(arr)))
print('Success')
conn.commit()
cur.close()
conn.close()


100%|██████████| 8001/8001 [02:19<00:00, 57.44it/s]

Success


In [17]:
conn.commit()
cur.close()
conn.close()
conn = psycopg2.connect(database="studs", user="s311745",
                        password="IpcU3kSHTeN94A3b", host="localhost", port="5432")
cur = conn.cursor()
cur.execute('''select * from Human''')
arr = cur.fetchall()
conn.commit()
cur.close()
conn.close()

In [18]:
arr

[]

In [ ]:
Research_center_id_arr

люди = работники + поставщики (работники > поставщики)
кадры > карта, области > карта, области <= кадры, буровые установки >= области, анализ месторождения = области
исслед. центер - мало



In [1]:
cur.close()
conn.close()

NameError: name 'cur' is not defined

In [1]:
print('AAA')

AAA


In [1]:
human_arr = employee_arr = supplier_arr = Research_center_id_arr = Map_arr = Shots_of_the_terrain_arr =\
 Analysis_of_deposits_id_arr = area_arr = []

In [8]:
human_arr.append(1)

In [12]:
human_arr

[1, 2, 4, 1]

In [2]:
round(0.3*50)

15